In [2]:
import sys
!{sys.executable} -m pip install geocoder

In [11]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [12]:
from bs4 import BeautifulSoup # BeautifulSoup is in bs4 package 
import requests
import pandas as pd
import numpy as np
import geocoder # import geocoder

In [13]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
content = requests.get(URL)

soup = BeautifulSoup(content.text, 'html.parser')

column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighbor_df = pd.DataFrame(columns=column_names)
postalCodes = [];
boroughs= [];
neighbourhoods = [];
        

In [14]:
first_table = soup.select_one("table:nth-of-type(1)")
for tr in first_table.find_all('tr'):
    cols = tr.find_all('td') 
    for col in cols:
        coltext = col.get_text()
        #get first 3 letters in postalcode
        postalCode = coltext[0:4]
        postalCode = postalCode.lstrip('\n')
        #split by ( and take first part as borough
        borough = coltext[4:].split("(", 1)[0]
        if borough.strip() != "" and borough.strip() != "Not assigned":
            #take 2nd part of split as neighbourhood
            neighbourhood = coltext[4:].split("(", 1)[1]
            neighbourhood = neighbourhood.rstrip(')\n')
            if neighbourhood == "Not assigned":
                neighbourhood = borough
                 
            #add values to arrays
            postalCodes.append(postalCode) 
            boroughs.append(borough)
            neighbourhoods.append(neighbourhood)

In [15]:
for data in range(len(neighbourhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighbourhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbor_df = neighbor_df.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [18]:
neighbor_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.723270,-79.451286
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661150,-79.391715
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653760,-79.510890
99,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.648690,-79.385440
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.632835,-79.489550
